In [1]:
!pip install "numpy<2.0" -q

In [ ]:
import os
import shutil
import glob
import yaml
import torch
from tqdm import tqdm

# ==========================================
# 1. SETUP & INSTALL
# ==========================================
print("--- STEP 1: Setup ---")
# Install Ultralytics quietly
os.system('pip install ultralytics -q')
print("✅ Ultralytics installed.")

from ultralytics import YOLO

# Check GPU
if torch.cuda.is_available():
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ WARNING: No GPU detected. Enable it in Session Options.")

# ==========================================
# 2. FIND & COPY DATASET
# ==========================================
print("\n--- STEP 2: Preparing Data ---")

# Define paths
INPUT_ROOT = '/kaggle/input'
WORKING_DIR = '/kaggle/working/my_dataset'

# Clean up previous runs
if os.path.exists(WORKING_DIR):
    shutil.rmtree(WORKING_DIR)

# 1. Search for data.yaml to find the dataset root
print("🔍 Searching for dataset folder...")
yaml_files = glob.glob(f"{INPUT_ROOT}/**/data.yaml", recursive=True)

if not yaml_files:
    print("❌ Error: Could not find 'data.yaml' in /kaggle/input.")
    print("Please check your 'Input' sidebar. Did you upload the dataset correctly?")
    raise FileNotFoundError("Dataset not found!")

# 2. Identify the Source Folder
src_yaml_path = yaml_files[0]
src_dataset_dir = os.path.dirname(src_yaml_path) # The folder containing data.yaml
print(f"✅ Found dataset at: {src_dataset_dir}")

# 3. Copy to Writable Directory (Required for YOLO caching & editing yaml)
print(f"📂 Copying data to {WORKING_DIR} (This may take a minute)...")
shutil.copytree(src_dataset_dir, WORKING_DIR)
print("✅ Copy Complete.")

# ==========================================
# 3. FIX DATA.YAML PATHS & CLASSES
# ==========================================
print("\n--- STEP 3: Fixing Config & Classes ---")

dest_yaml_path = os.path.join(WORKING_DIR, "data.yaml")

# 1. Load yaml
with open(dest_yaml_path, 'r') as f:
    data = yaml.safe_load(f)
    old_names = data['names']

# 2. FIX PATHS (Critical Step)
# We overwrite relative paths with ABSOLUTE paths to the new writable folder
data['path']  = WORKING_DIR 
data['train'] = "train/images"
data['val']   = "valid/images" # Roboflow uses 'valid', YOLO uses 'val'
data['test']  = "test/images"

# 3. Define Class Mapping
BIO_NAMES = ['biodegradable', 'cardboard', 'paper']
NON_BIO_NAMES = ['glass', 'metal', 'plastic', 'trash']

id_map = {}
print(f"Mapping from: {old_names}")

for idx, name in enumerate(old_names):
    clean_name = name.lower().strip()
    if clean_name in BIO_NAMES:
        id_map[idx] = 0 # Biodegradable
    else:
        id_map[idx] = 1 # Non-Biodegradable (Default)

# 4. Rewrite Label Files
label_files = glob.glob(f"{WORKING_DIR}/**/*.txt", recursive=True)
print(f"Processing {len(label_files)} label files...")

for file_path in tqdm(label_files):
    if file_path.endswith("classes.txt"): continue
    
    new_lines = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        parts = line.strip().split()
        if not parts: continue
        
        try:
            old_id = int(parts[0])
            if old_id in id_map:
                new_id = id_map[old_id]
                new_lines.append(f"{new_id} {' '.join(parts[1:])}\n")
        except ValueError:
            continue
            
    with open(file_path, 'w') as f:
        f.writelines(new_lines)

# 5. Update YAML with new Classes
data['nc'] = 2
data['names'] = ['Biodegradable', 'Non-Biodegradable']

with open(dest_yaml_path, 'w') as f:
    yaml.dump(data, f)

print("✅ Configuration Fixed!")

# ==========================================
# 4. START TRAINING
# ==========================================
print("\n--- STEP 4: Training YOLO11 ---")

PROJECT_DIR = '/kaggle/working/runs'
NAME = 'waste_detection'

model = YOLO('yolo11n.pt') 

results = model.train(
    data=dest_yaml_path, # Point to our fixed YAML
    epochs=50,          
    imgsz=640,
    batch=16,           
    project=PROJECT_DIR,
    name=NAME,
    patience=15,
    save=True,
    exist_ok=True
)

# ==========================================
# 5. ZIP RESULTS
# ==========================================
import shutil
from IPython.display import FileLink

print("\n--- STEP 5: Zipping Output ---")
output_folder = f"{PROJECT_DIR}/{NAME}"
zip_name = "YOLO_Results"
shutil.make_archive(zip_name, 'zip', output_folder)

print(f"✅ Zip created! Click below to download:")
display(FileLink(f"{zip_name}.zip"))

--- STEP 1: Setup ---


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
ydata-profiling 4.17.0 requires numpy<2.2,>=1.16.0, but you have numpy 2.2.6 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2

✅ Ultralytics installed.
✅ GPU Detected: Tesla T4

--- STEP 2: Preparing Data ---
🔍 Searching for dataset folder...
✅ Found dataset at: /kaggle/input/yolo-dataset
📂 Copying data to /kaggle/working/my_dataset (This may take a minute)...
✅ Copy Complete.

--- STEP 3: Fixing Config & Classes ---
Mapping from: ['BIODEGRADABLE', 'CARDBOARD', 'GLASS', 'METAL', 'PAPER', 'PLASTIC']
Processing 17367 label files...


100%|██████████| 17367/17367 [00:01<00:00, 10101.43it/s]


✅ Configuration Fixed!

--- STEP 4: Training YOLO11 ---
Ultralytics 8.3.237 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/my_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=waste_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=a

In [8]:
from ultralytics import YOLO

# Load the last checkpoint
model = YOLO("/kaggle/working/runs/waste_detection/weights/last.pt")

# Resume! (No need to set epochs, it remembers)
results = model.train(resume=True)

Ultralytics 8.3.237 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/my_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/runs/waste_detection/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=waste_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 66/66 4.7it/s 14.2s0.2s
                   all       2098      18916       0.76      0.599      0.692      0.455

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      33/50      5.04G      1.179      1.044      1.351          1        640: 100% ━━━━━━━━━━━━ 890/890 5.9it/s 2:32<0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 66/66 4.7it/s 14.2s0.2s
                   all       2098      18916      0.751      0.603      0.691      0.456

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      34/50      5.04G      1.183      1.035      1.339          6        640: 100% ━━━━━━━━━━━━ 890/890 5.8it/s 2:33<0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 66/66 4.7it/s 14.0s0.2s
                   all

In [12]:
from ultralytics import YOLO

# 1. Load the weights you just finished training
# (This file contains all the knowledge from the first 50 epochs)
model = YOLO("/kaggle/working/runs/waste_detection/weights/last.pt")

print("🚀 Starting Fine-Tuning...")

# 2. Start a NEW training run
# We do NOT use 'resume=True'. We treat it as a new job starting with a smart brain.
results = model.train(
    data='/kaggle/working/my_dataset/data.yaml',  # Point to your dataset config
    epochs=50,                  # Train for 50 MORE epochs (Total will be ~100)
    imgsz=640,
    batch=16,
    project='/kaggle/working/runs',
    name='waste_detection_v2',  # Save to a new folder so we don't overwrite the old one
    save=True,
    exist_ok=True
)

# 3. Zip the new results
import shutil
from IPython.display import FileLink

print("\n📦 Zipping new results...")
shutil.make_archive('YOLO_Results_v2', 'zip', '/kaggle/working/runs/waste_detection_v2')
print("✅ Done! Download below:")
display(FileLink('YOLO_Results_v2.zip'))

🚀 Starting Fine-Tuning...
Ultralytics 8.3.237 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/my_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/runs/waste_detection/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=waste_detection_v2, nbs=64, nms=False, opset=None, optimize=Fal

/kaggle/working/YOLO_Results_v2.zip

In [10]:
# ==========================================
# 5. ZIP RESULTS
# ==========================================
import shutil
from IPython.display import FileLink

print("\n--- STEP 5: Zipping Output ---")
output_folder = f"{PROJECT_DIR}/{NAME}"
zip_name = "YOLO_Results"
shutil.make_archive(zip_name, 'zip', output_folder)

print(f"✅ Zip created! Click below to download:")
display(FileLink(f"{zip_name}.zip"))


--- STEP 5: Zipping Output ---
✅ Zip created! Click below to download:


/kaggle/working/YOLO_Results.zip

In [ ]:
!yolo export model=/content/best.pt format=tflite int8